In [1]:
%load_ext autoreload
%autoreload 2

import os
import warnings

from IPython.core.debugger import set_trace
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import display, Markdown, HTML, Javascript
from tqdm import tqdm


def alert(message='Loop completed!'):
    # Check if the system supports 'say' and notifications
    os.system(f'osascript -e \'display notification "{message}" with title "Notification"\'')
    # Speak the alert
    os.system(f'say "{message}"')        
        
os.environ['PYTHONBREAKPOINT'] = 'IPython.core.debugger.set_trace'
breakpoint = set_trace
InteractiveShell.ast_node_interactivity = "all"
warnings.filterwarnings("ignore")
tqdm.pandas()

In [2]:
import json
import pandas as pd
import requests
import re
import time

from bs4 import BeautifulSoup
from datetime import datetime
from functools import lru_cache

# Find Trump Speeches

In [3]:
class TranscriptScraper:
    def __init__(self):
        self.base_url = 'https://www.rev.com/transcripts'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.transcripts = []
        
    def _extract_page_number(self, url):
        """Extract page number from URL parameter."""
        match = re.search(r'page=(\d+)', url)
        return int(match.group(1)) if match else 1

    def _get_next_page_url(self, soup):
        """Find the next page URL from the pagination element."""
        next_link = soup.find('a', {'class': 'w-pagination-next'})
        return next_link.get('href') if next_link else None

    def _extract_transcript_info(self, card):
        """Extract information from a single transcript card."""
        try:
            # Extract link
            link_elem = card.find('a', class_='blog-article-card_title-link')
            if not link_elem:
                return None
                
            relative_link = link_elem.get('href')
            full_link = f"https://www.rev.com{relative_link}"

            # Extract other elements
            title_elem = card.find('h3', class_='blog-article-card_heading')
            desc_elem = card.find('p')
            date_elem = card.find('div', class_='blog-article-card_meta')
            img_elem = card.find('img', class_='blog-article-card_img')

            return {
                'title': title_elem.text.strip() if title_elem else 'No title',
                'url': full_link,
                'description': desc_elem.text.strip() if desc_elem else 'No description',
                'date': date_elem.text.strip() if date_elem else 'No date',
                'image_url': img_elem.get('src') if img_elem else None
            }
        except KeyboardInterrupt:
            raise
        except Exception as e:
            print(f"Error processing card: {str(e)}")
            return None

    def validate_transcript(self, transcript):
        if not transcript:
            return False
        elif 'Trump' not in transcript['title']:
            return False
        elif 'speech' not in transcript['title'].lower():
            return False
        else:
            return True

    @lru_cache(maxsize=1000)
    def fetch_page(self, url):
        """Fetch and process a single page of transcripts."""
        try:
            response = requests.get(url, headers=self.headers)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            cards = soup.find_all('div', class_='blog-article-card_component')
            
            page_transcripts = []
            for card in cards:
                transcript = self._extract_transcript_info(card)
                if transcript:
                    page_transcripts.append(transcript)
                    
            return page_transcripts, self._get_next_page_url(soup)
            
        except KeyboardInterrupt:
            raise
        except requests.RequestException as e:
            print(f"Error fetching page {url}: {str(e)}")
            return [], None

    def fetch_transcripts(self, target_count=100, delay=1):
        """
        Fetch transcripts until reaching target count or no more pages.
        
        Args:
            target_count (int): Number of transcripts to collect before stopping
            delay (int): Delay between requests in seconds
        """
        current_url = self.base_url
        page_number = 1
        
        print(f"Starting transcript collection (target: {target_count})...")
        
        while len(self.transcripts) < target_count:
            print(f"\nFetching page {page_number}...")
            
            page_transcripts, next_page_url = self.fetch_page(current_url)
            
            if not page_transcripts:
                print("No transcripts found on this page.")
                break

            self.transcripts.extend([t for t in page_transcripts if self.validate_transcript(t)])
            print(f"Found {len(page_transcripts)} transcripts on page {page_number}")
            print(f"Total transcripts collected: {len(self.transcripts)}")
            
            if not next_page_url:
                print("\nNo more pages available.")
                break
                
            current_url = f"https://www.rev.com/transcripts{next_page_url}"
            page_number += 1
            
            # Respect the website by waiting between requests
            if delay:
                time.sleep(delay)
                
        return self.transcripts[:target_count]  # Return only up to target_count

    def save_transcripts(self, filename='transcripts.json'):
        """Save collected transcripts to a JSON file."""
        if self.transcripts:
            try:
                with open(filename, 'w', encoding='utf-8') as f:
                    json.dump(self.transcripts, f, indent=4, ensure_ascii=False)
                print(f"\nSuccessfully saved {len(self.transcripts)} transcripts to {filename}")
            except Exception as e:
                print(f"Error saving transcripts: {str(e)}")

In [25]:
# Initialize scraper
scraper = TranscriptScraper()

# Set target number of transcripts to collect
TARGET_TRANSCRIPTS = 50  # Adjust this number as needed

# Fetch transcripts
transcripts = scraper.fetch_transcripts(target_count=TARGET_TRANSCRIPTS)

Starting transcript collection (target: 50)...

Fetching page 1...
Found 9 transcripts on page 1
Total transcripts collected: 0

Fetching page 2...
Found 9 transcripts on page 2
Total transcripts collected: 0

Fetching page 3...
Found 9 transcripts on page 3
Total transcripts collected: 0

Fetching page 4...
Found 9 transcripts on page 4
Total transcripts collected: 0

Fetching page 5...
Found 9 transcripts on page 5
Total transcripts collected: 0

Fetching page 6...
Found 9 transcripts on page 6
Total transcripts collected: 0

Fetching page 7...
Found 9 transcripts on page 7
Total transcripts collected: 0

Fetching page 8...
Found 9 transcripts on page 8
Total transcripts collected: 0

Fetching page 9...
Found 9 transcripts on page 9
Total transcripts collected: 0

Fetching page 10...
Found 9 transcripts on page 10
Total transcripts collected: 0

Fetching page 11...
Found 9 transcripts on page 11
Total transcripts collected: 0

Fetching page 12...
Found 9 transcripts on page 12
Total 

# Download and Parse Trump Speeches

In [4]:
import unicodedata


TIMESTAMP_REGEX = re.compile(r'(\(|\[)\s*(inaudible|crosstalk)?\s*(\d{2}:\d{2}(:\d{2})?)\s*(\)|\])')
SPEAKER_REGEX = re.compile(r'((((Jr\.|Sen\.|Rep\.|[A-Z]+[a-z]*|\d)\s?){1,5}):?\s*\(\s*(\d{2}:\d{2}(:\d{2})?)\s*\):?\s*)')

def clean_paragraph(p):
    return p.strip().replace('\n', '').encode('ascii', 'ignore').decode('ascii')        

def parse_timestamp(timestamp_text):
    """Extract timestamp from format like (00:00) or (01:23)"""
    if not timestamp_text:
        return None
    match = re.search(TIMESTAMP_REGEX, timestamp_text)
    return match.group(3) if match else None

def parse_transcript_content(content):
    """Parse transcript content into structured format"""
    soup = BeautifulSoup(content, 'html.parser')
    content_div = soup.find('div', class_='blog-text-rich-text')
    paragraphs = re.sub(SPEAKER_REGEX, r'\r\1\r', content_div.get_text()).split('\r')
        
    transcript_data = []
    current_speaker = None
    current_timestamp = None
    
    for i, p in enumerate(paragraphs):
        text = clean_paragraph(p)
        if not text:
            continue
            
        # Look for speaker pattern (Name (00:00):)
        speaker_match = re.match(SPEAKER_REGEX, text)
        
        if speaker_match:
            current_speaker = speaker_match.group(2).strip()
            current_timestamp = parse_timestamp(text) or current_timestamp
            continue
            
        # Look for timestamp in text
        timestamp = parse_timestamp(text) or current_timestamp
        utterance = re.sub(TIMESTAMP_REGEX, '', text).strip(': ')
        
        if current_speaker and len(utterance) > 1:
            transcript_data.append({
                'speaker': current_speaker,
                'timestamp': timestamp,
                'utterance': utterance
            })

        current_timestamp = None
        
    return pd.DataFrame(transcript_data)

In [5]:
from functools import lru_cache

@lru_cache(maxsize=int(10e4))
def get_transcript_content(url):
    """Get content of individual transcript page"""
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        content_div = soup.find('div', class_='blog-text-rich-text')
        
        if content_div:
            return content_div.prettify()
        return None
    except Exception as e:
        print(f"Error getting transcript content from {url}: {str(e)}")
        return None

In [6]:
timestamp = datetime.now().strftime("%Y%m%d")

In [7]:
from extract_from_speech_txt import chunk_into_sentences

trump_output_filename = f'trump_speeches_{timestamp}.csv'

def process_df(df):
    rows = df[df.text.notna()].to_dict(orient='records')
    new_rows = []
    for row in rows:
        sentences = chunk_into_sentences(row.pop('utterance'))
        new_rows.extend([{**row, 'text': sentence.strip()} for sentence in sentences if sentence.strip()])
    
    filename = f'data/raw/trump_speech{i+1}.csv'

    df = pd.DataFrame(new_rows)
    df.to_csv(filename, index=False)
    process_transcript(filename, f'data/processed/{trump_output_filename}', speaker='TRUMP', speech_mode=True)

In [123]:
from extract_from_remarks_csv import process_transcript

# Process each transcript
all_transcripts = []
for i, transcript in enumerate(transcripts):
    url = transcript['url']
    print(f"\nProcessing transcript {i+1}/{len(transcripts)}")
    print(f"URL: {url}")
    
    content = get_transcript_content(url)
    if content:
        try:
            df = parse_transcript_content(content)
            for k, v in transcript.items():
                df[k] = v
            df['text'] = df['utterance']
            df['source'] = url
            
            all_transcripts.append(df)
            process_df(df)
            
            print(f"Successfully parsed {len(df)} utterances")
        except Exception as e:
            raise
            print(f"Error parsing transcript: {str(e)}")
    
    # Be nice to the server
    # time.sleep(1)

# Combine all transcripts
final_df = pd.concat(all_transcripts, ignore_index=True)
print(f"\nTotal utterances parsed: {len(final_df)}")


Processing transcript 1/50
URL: https://www.rev.com/transcripts/trump-victory-speech
Successfully parsed 19 utterances

Processing transcript 2/50
URL: https://www.rev.com/transcripts/donald-trump-victory-speech-after-winning-south-carolina-gop-primary-transcript
Successfully parsed 16 utterances

Processing transcript 3/50
URL: https://www.rev.com/transcripts/donald-trump-delivers-speech-in-south-carolina-transcript-e69f4
Successfully parsed 45 utterances

Processing transcript 4/50
URL: https://www.rev.com/transcripts/donald-trump-delivers-speech-in-south-carolina-transcript
Successfully parsed 45 utterances

Processing transcript 5/50
URL: https://www.rev.com/transcripts/donald-trump-delivers-speech-in-iowa-transcript
Successfully parsed 4 utterances

Processing transcript 6/50
URL: https://www.rev.com/transcripts/former-us-president-donald-trump-delivers-speech-ahead-of-independence-day-transcript
Successfully parsed 16 utterances

Processing transcript 7/50
URL: https://www.rev.c

In [124]:
trump_speech_df = pd.read_csv(f'data/processed/{trump_output_filename}')
for col in ['content', 'utterance', 'full_text']:
    trump_speech_df[col] = trump_speech_df.apply(lambda x: x[col].replace(x.speaker.strip().upper(), 'MAIN SPEAKER'), axis=1)

trump_speech_df.to_csv('data/processed/trump_speeches_processed.csv', index=False)

In [125]:
trump_speech_df

,speaker,content,content_tokens,utterance,utterance_tokens,full_text,source
0,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,13,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,12,Thank you very much. Wow. USA. USA. USA. USA. ...,https://www.rev.com/transcripts/trump-victory-...
1,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,15,<|begin_of_text|>Wow.,3,Thank you very much. Wow. USA. USA. USA. USA. ...,https://www.rev.com/transcripts/trump-victory-...
2,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,38,"<|begin_of_text|>Well, I want to thank you all...",12,Thank you very much. Wow. USA. USA. USA. USA. ...,https://www.rev.com/transcripts/trump-victory-...
3,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,42,<|begin_of_text|>This is great.,5,Thank you very much. Wow. USA. USA. USA. USA. ...,https://www.rev.com/transcripts/trump-victory-...
4,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Thank you very ...,47,<|begin_of_text|>These are our friends.,6,Thank you very much. Wow. USA. USA. USA. USA. ...,https://www.rev.com/transcripts/trump-victory-...
...,...,...,...,...,...,...,...
25917,Donald Trump,"<|begin_of_text|>MAIN SPEAKER: So, we bring ou...",1019,<|begin_of_text|>We have made America wealthy ...,12,(singing). USA! USA! USA! USA! USA! USA! USA! ...,https://www.rev.com/transcripts/donald-trump-r...
25918,Donald Trump,<|begin_of_text|>MAIN SPEAKER: We've been incr...,1026,<|begin_of_text|>We have made America strong a...,16,(singing). USA! USA! USA! USA! USA! USA! USA! ...,https://www.rev.com/transcripts/donald-trump-r...
25919,Donald Trump,<|begin_of_text|>MAIN SPEAKER: Nobody has our ...,1029,<|begin_of_text|>We have made America proud ag...,15,(singing). USA! USA! USA! USA! USA! USA! USA! ...,https://www.rev.com/transcripts/donald-trump-r...
25920,Donald Trump,<|begin_of_text|>MAIN SPEAKER: ... but we aren...,1029,<|begin_of_text|>We have made America safe aga...,16,(singing). USA! USA! USA! USA! USA! USA! USA! ...,https://www.rev.com/transcripts/donald-trump-r...


# Download and parse inaugural speeches

In [126]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

@lru_cache(maxsize=1000)
def get_inaugural_speech(i):
    """Get inaugural address for a given index"""
    url = f"https://www.presidency.ucsb.edu/documents/inaugural-address-{i}"
    
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Get president's name
        president_elem = soup.find('h3', class_='diet-title')
        president = president_elem.text.strip() if president_elem else None
        
        # Get speech date
        date_elem = soup.find('span', class_='date-display-single')
        date = date_elem.text.strip() if date_elem else None
        
        # Get speech content
        content_div = soup.find('div', class_='field-docs-content')
        if content_div:
            paragraphs = content_div.find_all('p')
            speech = ' '.join(p.text.strip() for p in paragraphs)
        else:
            speech = None
            
        return {
            'president': president,
            'date': date,
            'speech': speech,
            'url': url
        }
        
    except Exception as e:
        print(f"Error fetching speech {i}: {str(e)}")
        return None

In [127]:
inaugural_output_filename = 'inaugural_speeches_20250126.csv'
trump_inaugural_output_filename = 'trump_inaugural_speech_20250126.csv'

In [128]:
from extract_from_speech_txt import extract_remarks_from_speech
from extract_from_remarks_csv import process_transcript


all_speeches = []

for i in range(1, 55):  # 1 to 54
    print(f"\nFetching inaugural address {i}...")
    speech_data = get_inaugural_speech(i)

    speech = f'data/raw/speech{i}.txt'
    with open(speech, 'w') as f:
        f.write(speech_data['speech'])
    filename = f'data/raw/speech{i}.csv'
 
    remarks = extract_remarks_from_speech(speech, speaker='MAIN SPEAKER')
    df = pd.DataFrame(remarks)
    df['source'] = f"https://www.presidency.ucsb.edu/documents/inaugural-address-{i}"
    df.to_csv(filename, index=False)
    if i == 54:
        process_transcript(filename, f'data/processed/{trump_inaugural_output_filename}', speaker='MAIN SPEAKER', speech_mode=True)
    else:
        process_transcript(filename, f'data/processed/{inaugural_output_filename}', speaker='MAIN SPEAKER', speech_mode=True)

    if speech_data:
        all_speeches.append(speech_data)
        print(f"Successfully retrieved speech by {speech_data['president']}")
    
    # Be polite to the server
    # time.sleep(2)

# Create DataFrame
df = pd.DataFrame(all_speeches)

# Save to CSV
df.to_csv('data/raw/inaugural_addresses.csv', index=False)
print(f"\nSaved {len(df)} speeches to inaugural_addresses.csv")


Fetching inaugural address 1...


11508

Successfully retrieved speech by Richard Nixon

Fetching inaugural address 2...


7541

Successfully retrieved speech by John F. Kennedy

Fetching inaugural address 3...


...

Successfully retrieved speech by George W. Bush

Fetching inaugural address 53...


13889

Successfully retrieved speech by Joseph R. Biden, Jr.

Fetching inaugural address 54...


17538

Successfully retrieved speech by Donald J. Trump (2nd Term)

Saved 54 speeches to inaugural_addresses.csv


In [129]:
trump_inaugural_speech_df = pd.read_csv(f'data/processed/{trump_inaugural_output_filename}')
for col in ['content', 'utterance', 'full_text']:
    trump_inaugural_speech_df[col] = trump_inaugural_speech_df.apply(lambda x: x[col].replace(x.speaker.strip().upper(), 'MAIN SPEAKER'), axis=1)

trump_inaugural_speech_df.to_csv(f'data/processed/trump_inaugural_speech.csv', index=False)

In [138]:
inaugural_speech_df = pd.read_csv(f'data/processed/{inaugural_output_filename}')
for col in ['content', 'utterance', 'full_text']:
    inaugural_speech_df[col] = inaugural_speech_df.apply(lambda x: x[col].replace(x.speaker.strip().upper(), 'MAIN SPEAKER'), axis=1)

inaugural_speech_df.to_csv(f'data/processed/inaugural_speeches_processed.csv', index=False)

# Reformat Trump Tweets and Concatenate to Speeches

In [134]:
from extract_from_remarks_csv import get_token_length


tweet_df = pd.read_csv('data/processed/trump_tweets_processed.csv')
speech_df = pd.read_csv('data/processed/trump_speeches_processed.csv')

BOS_TOKEN = '<|begin_of_text|>'
EOS_TOKEN = '<|end_of_text|>'
tweet_df['speaker'] = 'Donald Trump'
tweet_df['source'] = 'https://x.com/'
tweet_df['utterance'] = tweet_df.utterance.str.replace('MAIN SPEAKER: ', '')
tweet_df['full_text'] = 'MAIN SPEAKER: ' + tweet_df.utterance.str.removeprefix(BOS_TOKEN).str.removesuffix(EOS_TOKEN)
tweet_df['utterance'] = BOS_TOKEN + tweet_df.full_text
tweet_df['content'] = BOS_TOKEN + tweet_df.full_text + EOS_TOKEN
tweet_df['content_tokens'] = tweet_df.content.apply(get_token_length)
tweet_df['utterance_tokens'] = tweet_df.utterance.apply(get_token_length)

In [135]:
pd.concat([tweet_df, speech_df]).to_csv('data/processed/trump_all_processed.csv', index=False)
tweet_df.to_csv('data/processed/trump_posts_processed.csv', index=False)

In [137]:
pd.concat([tweet_df, speech_df]).utterance.iloc[0]
pd.concat([tweet_df, speech_df]).utterance.iloc[-1]

'<|begin_of_text|>MAIN SPEAKER: I will be on @FoxNews live,  with members of my family, at 11:50 P.M. We will ring in the New Year together! MAKE AMERICA GREAT AGAIN!'

'<|begin_of_text|>Go get him, David, go get him, Kelly, go get them tomorrow.'